# Работа с моделями семейства Meta

## Введение

В этом уроке мы рассмотрим:

- Два основных представителя семейства Meta — Llama 3.1 и Llama 3.2
- Сценарии и задачи, для которых подходит каждая из моделей
- Пример кода, демонстрирующий уникальные возможности каждой модели

## Семейство моделей Meta

В этом уроке мы познакомимся с двумя моделями из семейства Meta, также известного как "Llama Herd" — Llama 3.1 и Llama 3.2

Эти модели представлены в нескольких вариантах и доступны на маркетплейсе моделей Github. Подробнее о том, как использовать Github Models для [прототипирования с AI-моделями](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst).

Варианты моделей:
- Llama 3.1 — 70B Instruct
- Llama 3.1 — 405B Instruct
- Llama 3.2 — 11B Vision Instruct
- Llama 3.2 — 90B Vision Instruct

*Примечание: Llama 3 также доступна на Github Models, но в этом уроке она не рассматривается*


## Llama 3.1

С 405 миллиардами параметров Llama 3.1 относится к категории открытых LLM.

Эта версия является улучшением по сравнению с предыдущей Llama 3 и предлагает:

- Более широкое контекстное окно — 128k токенов против 8k токенов
- Большее максимальное количество выходных токенов — 4096 против 2048
- Улучшенная поддержка разных языков — благодаря увеличению объёма обучающих данных

Это позволяет Llama 3.1 справляться с более сложными задачами при создании GenAI-приложений, включая:
- Вызов функций на уровне модели — возможность обращаться к внешним инструментам и функциям вне стандартного процесса LLM
- Улучшенная производительность RAG — благодаря увеличенному контекстному окну
- Генерация синтетических данных — возможность создавать эффективные данные для задач, таких как дообучение


### Вызов нативных функций

Llama 3.1 была дообучена для более эффективного использования функций и инструментов. В ней также есть два встроенных инструмента, которые модель может определить как необходимые для использования в зависимости от запроса пользователя. Эти инструменты:

- **Brave Search** — может использоваться для получения актуальной информации, например, прогноза погоды, с помощью веб-поиска
- **Wolfram Alpha** — подходит для более сложных математических вычислений, поэтому нет необходимости писать собственные функции.

Вы также можете создавать свои собственные инструменты, которые LLM сможет вызывать.

В примере кода ниже:

- Мы определяем доступные инструменты (brave_search, wolfram_alpha) в системном промпте.
- Отправляем пользовательский запрос о погоде в определённом городе.
- LLM ответит вызовом инструмента Brave Search, который будет выглядеть так: `<|python_tag|>brave_search.call(query="Stockholm weather")`

*Примечание: В этом примере происходит только вызов инструмента. Если вы хотите получить результаты, вам нужно создать бесплатный аккаунт на странице Brave API и определить саму функцию.*


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "meta-llama-3.1-405b-instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)


tool_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant<|eot_id|>
"""

messages = [
    SystemMessage(content=tool_prompt),
    UserMessage(content="What is the weather in Stockholm?"),

]

response = client.complete(messages=messages, model=model_name)

print(response.choices[0].message.content)





### Llama 3.2

Несмотря на то, что Llama 3.1 является LLM, у неё есть ограничение — отсутствие мультимодальности. То есть, она не может использовать разные типы входных данных, например, изображения, в качестве подсказок и давать на них ответы. Эта возможность стала одной из ключевых особенностей Llama 3.2. Среди новых функций также:

- Мультимодальность — умеет обрабатывать как текстовые, так и визуальные подсказки
- Варианты малых и средних размеров (11B и 90B) — это даёт гибкость при развертывании,
- Только текстовые варианты (1B и 3B) — позволяют запускать модель на мобильных и edge-устройствах с низкой задержкой

Поддержка мультимодальности — это значительный шаг вперёд для мира open source моделей. В примере кода ниже используется и изображение, и текстовая подсказка, чтобы получить анализ изображения от Llama 3.2 90B.

### Мультимодальная поддержка в Llama 3.2


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "Llama-3.2-90B-Vision-Instruct"


client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(
            content="You are a helpful assistant that describes images in details."
        ),
        UserMessage(
            content=[
                TextContentItem(text="What's in this image?"),
                ImageContentItem(
                    image_url=ImageUrl.load(
                        image_file="sample.jpg",
                        image_format="jpg",
                        detail=ImageDetailLevel.LOW)
                ),
            ],
        ),
    ],
    model=model_name,
)

print(response.choices[0].message.content)

## На этом обучение не заканчивается — продолжайте свой путь

После завершения этого урока загляните в нашу [подборку по обучению генеративному ИИ](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst), чтобы продолжить развивать свои знания в области генеративного искусственного интеллекта!



---

**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Несмотря на наши усилия обеспечить точность, автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на исходном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные толкования, возникшие в результате использования данного перевода.
